### 1- Order Details Table

In [ ]:

import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Define the date range filter (1/1/2025 to 31/1/2025)
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2025-03-10')
]

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch orders
    print("Fetching POS orders...")
    orders = fetch_data_in_bulk('pos.order', ['name', 'date_order', 'lines'], date_domain, uid)

    if not orders:
        print("No orders fetched. Exiting.")
        return
    print(f"Fetched {len(orders)} orders.")

    # Step 3: Prefetch all order lines and products
    print("Fetching related order lines and products...")
    all_line_ids = [line_id for order in orders for line_id in order.get('lines', [])]
    order_lines = fetch_data_in_bulk('pos.order.line', ['id', 'price_unit', 'qty', 'product_id'], [['id', 'in', all_line_ids]], uid)

    all_product_ids = list({line.get('product_id', [])[0] for line in order_lines if line.get('product_id')})
    products = fetch_data_in_bulk('product.product', ['id', 'display_name', 'standard_price'], [['id', 'in', all_product_ids]], uid)

    # Step 4: Map products for faster lookups
    product_map = {product['id']: product for product in products}

    # Step 5: Prepare and batch insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'order_details'
        )
        BEGIN
            CREATE TABLE order_details (
                id INT IDENTITY(1,1) PRIMARY KEY,
                order_ref NVARCHAR(255),
                order_date DATETIME,
                unit_price DECIMAL(18, 2),
                quantity DECIMAL(18, 2),
                product_name NVARCHAR(255),
                product_cost DECIMAL(18, 2)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for order in orders:
            order_ref = order.get('name')
            order_date = order.get('date_order')
            line_ids = order.get('lines', [])

            for line_id in line_ids:
                line = next((l for l in order_lines if l['id'] == line_id), None)
                if not line:
                    continue

                unit_price = line.get('price_unit')
                quantity = line.get('qty')
                product_id = line.get('product_id', [])[0] if line.get('product_id') else None

                product_name, product_cost = None, None
                if product_id and product_id in product_map:
                    product = product_map[product_id]
                    product_name = product.get('display_name')
                    product_cost = product.get('standard_price')

                # Add to batch
                batch_data.append((order_ref, order_date, unit_price, quantity, product_name, product_cost))

                # Insert in batches
                if len(batch_data) >= batch_size:
                    cursor.executemany(f"""
                    INSERT INTO order_details (
                        order_ref, order_date, unit_price, quantity, product_name, product_cost
                    ) VALUES (%s, %s, %s, %s, %s, %s)
                    """, batch_data)
                    conn.commit()
                    batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO order_details (
                order_ref, order_date, unit_price, quantity, product_name, product_cost
            ) VALUES (%s, %s, %s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()
Starting data synchronization...
Authenticated successfully. User ID: 112
Fetching POS orders...
Fetched 111570 orders.
Fetching related order lines and products...
Inserting data into SQL Server...
All data inserted successfully.
 

### 2- Orders Table

In [ ]:

import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Define the date range filter (1/1/2025 to 31/1/2025)
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2025-03-10')
]

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch orders
    print("Fetching POS orders...")
    orders = fetch_data_in_bulk('pos.order', [
        'name',         # Order Ref
        'session_id',   # Session
        'date_order',   # Date
        'pos_reference',# Receipt Number
        'return_ref',   # Return Ref
        'partner_id',   # Customer
        'user_id',      # Employee
        'amount_total'  # Total
    ], date_domain, uid)

    if not orders:
        print("No orders fetched. Exiting.")
        return
    print(f"Fetched {len(orders)} orders.")

    # Step 3: Prepare and batch insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'orders'
        )
        BEGIN
            CREATE TABLE orders (
                id INT IDENTITY(1,1) PRIMARY KEY,
                order_ref NVARCHAR(255) NOT NULL,      -- Order Ref
                session NVARCHAR(255),                -- Session
                date DATETIME,                        -- Date
                receipt_number NVARCHAR(255),         -- Receipt Number
                return_ref NVARCHAR(255),             -- Return Ref
                customer NVARCHAR(255),               -- Customer
                employee NVARCHAR(255),               -- Employee
                total DECIMAL(18, 2)                  -- Total Amount
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for order in orders:
            order_ref = order.get('name')
            session = order.get('session_id', [])[1] if isinstance(order.get('session_id', []), list) and len(order.get('session_id', [])) > 1 else ''
            date = order.get('date_order')
            receipt_number = order.get('pos_reference')
            return_ref = order.get('return_ref', '')
            customer = order.get('partner_id', [])[1] if isinstance(order.get('partner_id', []), list) and len(order.get('partner_id', [])) > 1 else ''
            employee = order.get('user_id', [])[1] if isinstance(order.get('user_id', []), list) and len(order.get('user_id', [])) > 1 else ''
            total = order.get('amount_total')

            # Add to batch
            batch_data.append((order_ref, session, date, receipt_number, return_ref, customer, employee, total))

            # Insert in batches
            if len(batch_data) >= batch_size:
                cursor.executemany(f"""
                INSERT INTO orders (
                    order_ref, session, date, receipt_number, return_ref, 
                    customer, employee, total
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, batch_data)
                conn.commit()
                batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO orders (
                order_ref, session, date, receipt_number, return_ref, 
                customer, employee, total
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()
 

### 3- Payments Table

In [ ]:

import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Define the date range filter (1/1/2025 to 31/1/2025)
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2025-03-10')
]

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch orders
    print("Fetching POS orders...")
    orders = fetch_data_in_bulk('pos.order', ['name', 'state', 'payment_ids'], date_domain, uid)

    if not orders:
        print("No orders fetched. Exiting.")
        return
    print(f"Fetched {len(orders)} orders.")

    # Step 3: Prefetch all payments
    print("Fetching related payments...")
    all_payment_ids = [payment_id for order in orders for payment_id in order.get('payment_ids', [])]
    payments = fetch_data_in_bulk('pos.payment', ['id', 'display_name', 'payment_method_id'], [['id', 'in', all_payment_ids]], uid)

    # Step 4: Map payment methods for faster lookups
    payment_method_ids = list({payment.get('payment_method_id', [])[0] for payment in payments if payment.get('payment_method_id')})
    payment_methods = fetch_data_in_bulk('pos.payment.method', ['id', 'name'], [['id', 'in', payment_method_ids]], uid)
    payment_method_map = {method['id']: method.get('name') for method in payment_methods}

    # Step 5: Prepare and batch insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'payments'
        )
        BEGIN
            CREATE TABLE payments (
                id INT IDENTITY(1,1) PRIMARY KEY,
                order_ref NVARCHAR(255),
                status NVARCHAR(255),
                payment_display_name NVARCHAR(255),
                payment_method NVARCHAR(255)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for order in orders:
            order_ref = order.get('name')
            status = order.get('state')
            payment_ids = order.get('payment_ids', [])

            for payment_id in payment_ids:
                payment = next((p for p in payments if p['id'] == payment_id), None)
                if not payment:
                    continue

                payment_display_name = payment.get('display_name')
                payment_method_id = payment.get('payment_method_id', [])[0] if payment.get('payment_method_id') else None
                payment_method = payment_method_map.get(payment_method_id, None)

                # Add to batch
                batch_data.append((order_ref, status, payment_display_name, payment_method))

                # Insert in batches
                if len(batch_data) >= batch_size:
                    cursor.executemany(f"""
                    INSERT INTO payments (
                        order_ref, status, payment_display_name, payment_method
                    ) VALUES (%s, %s, %s, %s)
                    """, batch_data)
                    conn.commit()
                    batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO payments (
                order_ref, status, payment_display_name, payment_method
            ) VALUES (%s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()
 

### 4- Customer Table

In [6]:

import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting customer data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch customer data
    print("Fetching customer data...")
    customers = fetch_data_in_bulk('res.partner', ['id', 'name'], uid)

    if not customers:
        print("No customers fetched. Exiting.")
        return
    print(f"Fetched {len(customers)} customers.")

    # Step 3: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'customer'
        )
        BEGIN
            CREATE TABLE customer (
                id INT PRIMARY KEY,
                customer_name NVARCHAR(255)
            );
        END
        """)
        conn.commit()

        print("Inserting customer data into SQL Server...")
        batch_data = []

        for customer in customers:
            customer_id = customer.get('id')
            customer_name = customer.get('name')  # The name can be in Arabic or other languages

            # Add to batch
            batch_data.append((customer_id, customer_name))

            # Insert in batches
            if len(batch_data) >= batch_size:
                cursor.executemany(f"""
                INSERT INTO customer (id, customer_name)
                VALUES (%d, %s)
                """, batch_data)
                conn.commit()
                batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO customer (id, customer_name)
            VALUES (%d, %s)
            """, batch_data)
            conn.commit()

        print("All customer data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting customer data synchronization...
Authenticated successfully. User ID: 112
Fetching customer data...
Fetched 841 customers.
Inserting customer data into SQL Server...
All customer data inserted successfully.
